In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Layer, MaxPooling2D, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import cv2

In [6]:
def read_img(real_path, fake_path):
    # Đọc ảnh từ cả hai thư mục
    real_image_files = sorted([os.path.join(real_path, f) for f in os.listdir(real_path) if f.endswith('.png')])
    fake_image_files = sorted([os.path.join(fake_path, f) for f in os.listdir(fake_path) if f.endswith('.png')])

    # Gộp ảnh từ cả hai thư mục vào một danh sách duy nhất
    all_image_files = real_image_files + fake_image_files

    # Tạo nhãn tương ứng cho ảnh (0 cho real, 1 cho fake)
    all_labels = [0] * len(real_image_files) + [1] * len(fake_image_files)

    return all_image_files, all_labels



In [7]:
def load_and_preprocess_image(image_path, image_size):
    img = cv2.imread(image_path)
    img = cv2.resize(img, image_size)
    img = img / 255.0  
    return img

In [8]:
def img_to_batch(batch_size, all_image_files, all_labels):
    num_batch = len(all_image_files) // batch_size

    for i in range(num_batch):

        batch_files = all_image_files[i * batch_size:(i + 1) * batch_size]
        batch_labels = all_labels[i * batch_size:(i + 1) * batch_size]
        
        batch_images = [load_and_preprocess_image(f) for f in batch_files]
        batch_images = np.stack(batch_images, axis=0)
        
        batch_labels = np.array(batch_labels)

    return batch_images, batch_labels

In [9]:
#ResNet50 model
def extract_features(image_size, batch_image):
    #using resnet50 with imagenet weight
    extract_model = ResNet50(
        weights = 'imagenet',
        include_top = False,
        input_shape = image_size
    )

    output = extract_model.output
    output = MaxPooling2D(output)
    output = Flatten(output)

    #extract_feature
    features = extract_model.predict(batch_image)
    return features

In [ ]:
def combine(real_path, fake_path):
    all_image_files, all_labels = load_and_preprocess_image(real_path, fake_path)
    batch_image, batch_labels = img_to_batch(batch_size=16, all_image_files, all_labels)
    features = extract_features(batch_image, batch_labels)
    return features

In [ ]:
features = combine()